In [3]:
import random
import json
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import nltk 

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
intents = json.loads(open('intents.json').read())

In [15]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [16]:
words = [word for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

### **Calculate X**

In [17]:
question = []
for doc in intents['intents']:
  question.append(doc['patterns'])

In [18]:
question_list = []
arr = [[question_list.append(x) for x in q] for q in question]

In [19]:
question_list

['Hi there',
 'How are you',
 'How are you?',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day',
 'Bye',
 'See you later',
 'Goodbye',
 'Nice chatting to you, bye',
 'Till next time',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Awesome, thanks',
 'Thanks for helping me',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'who are you?',
 'how old',
 'old',
 'how old is florian',
 'what is your age',
 'old are you',
 'age?',
 'How you could help me?',
 'What you can do?',
 'What help you provide?',
 'How you can be helpful?',
 'What support is offered',
 'How to check Adverse drug reaction?',
 'Open adverse drugs module',
 'Give me a list of drugs causing adverse behavior',
 'List all drugs suitable for patient with adverse reaction',
 'Which drugs dont have adverse reaction?',
 'Open blood pressure module',
 'Task related to blood pressure',
 'Blood pressure data entry',
 'I want to log blood pressure results',
 'Blood pressure data management',
 'I want

### **Calculate Y**

In [20]:
tag = []
for doc in documents:
  tag.append(doc[1])

In [21]:
Y = pd.get_dummies(tag)

In [22]:
num_unique_tags = len(Y.nunique())

In [23]:
num_unique_tags

11

### **X and Y**

In [24]:
X_train = np.array(question_list)[0:50]
y_train = np.array(Y)[0:50]
X_test = np.array(question_list)[50:]
y_test = np.array(Y)[50:]

In [25]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

50
50
8
8


### **Fine Tune on BERT**

In [16]:
!pip install -q transformers

     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 39.1 MB/s 
     |████████████████████████████████| 163 kB 72.8 MB/s 


In [17]:
from transformers import AutoTokenizer, TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
train_encodings = tokenizer(
    text=X_train.tolist(),
    add_special_tokens=True,
    max_length=9,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

test_encodings = tokenizer(
    text=X_test.tolist(),
    add_special_tokens=True,
    max_length=9,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [19]:
train_encodings

{'input_ids': <tf.Tensor: shape=(50, 9), dtype=int32, numpy=
array([[  101,  8790,  1175,   102,     0,     0,     0,     0,     0],
       [  101,  1731,  1132,  1128,   102,     0,     0,     0,     0],
       [  101,  1731,  1132,  1128,   136,   102,     0,     0,     0],
       [  101,  2181,  2256,  1175,   136,   102,     0,     0,     0],
       [  101,  4403,   102,     0,     0,     0,     0,     0,     0],
       [  101,  9800,  1742,   102,     0,     0,     0,     0,     0],
       [  101,  8667,   102,     0,     0,     0,     0,     0,     0],
       [  101,  2750,  1285,   102,     0,     0,     0,     0,     0],
       [  101, 17774,   102,     0,     0,     0,     0,     0,     0],
       [  101,  3969,  1128,  1224,   102,     0,     0,     0,     0],
       [  101, 15938,   102,     0,     0,     0,     0,     0,     0],
       [  101,  8835, 25132,  1106,  1128,   117, 11901,   102,     0],
       [  101, 22430,  1397,  1159,   102,     0,     0,     0,     0],
   

In [20]:
y_train[0:10]

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [21]:
print(len(train_encodings['input_ids']))
print(len(y_train))

50
50


In [22]:
input_ids = train_encodings['input_ids']
attention_mask = train_encodings['attention_mask']

### **Model Building**

In [23]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [24]:
max_len = 9
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids, attention_mask=input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation = 'relu')(out)
y = Dense(num_unique_tags, activation='sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [25]:
optimizer = Adam(learning_rate=5e-05, epsilon=1e-08, decay=0.01, clipnorm=1.0)
loss =CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metric)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 9)]          0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 9)]          0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 9, 7                                           

In [27]:
train_history = model.fit(
    x = {'input_ids':train_encodings['input_ids'],'attention_mask':train_encodings['attention_mask']},
    y = y_train,
    validation_data = (
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']}, y_test
    ),
    epochs=30,
    batch_size=36
)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 51s 7s/step - loss: 2.5750 - balanced_accuracy: 0.0800 - val_loss: 2.6667 - val_balanced_accuracy: 0.0000e+00
Epoch 2/30
2/2 [==============================] - 7s 3s/step - loss: 2.2066 - balanced_accuracy: 0.2600 - val_loss: 2.7052 - val_balanced_accuracy: 0.0000e+00
Epoch 3/30
2/2 [==============================] - 7s 3s/step - loss: 1.9764 - balanced_accuracy: 0.4000 - val_loss: 2.6459 - val_balanced_accuracy: 0.0000e+00
Epoch 4/30
2/2 [==============================] - 7s 3s/step - loss: 1.7498 - balanced_accuracy: 0.5200 - val_loss: 2.6427 - val_balanced_accuracy: 0.0000e+00
Epoch 5/30
2/2 [==============================] - 7s 3s/step - loss: 1.5459 - balanced_accuracy: 0.5200 - val_loss: 2.5378 - val_balanced_accuracy: 0.0000e+00
Epoch 6/30
2/2 [==============================] - 7s 3s/step - loss: 1.3649 - balanced_accuracy: 0.6200 - val_loss: 2.4830 - val_balanced_accuracy: 0.0000e+00
Epoch 7/30
2/2 [==============================] - 7s 3s/

In [43]:
model.save('chatbot_BERT_model.h5')

### **Model Evaluation**

In [29]:
predicted_raw = model.predict({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']})
predicted_raw[0]

array([0.5725007 , 0.01528618, 0.04648018, 0.8230653 , 0.14540422,
       0.06456128, 0.13001108, 0.49097577, 0.57712996, 0.04708493,
       0.07117614], dtype=float32)

In [30]:
y_predicted = np.argmax(predicted_raw, axis=1)
y_true = [y.tolist().index(1) for y in y_test]

In [31]:
y_predicted

array([3, 9, 9, 9, 3, 2, 3, 2])

In [32]:
y_true

[9, 9, 9, 6, 6, 6, 6, 6]

In [33]:
print(len(y_predicted))
print(len(y_true))

8
8


In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         5
           9       0.67      0.67      0.67         3

    accuracy                           0.25         8
   macro avg       0.17      0.17      0.17         8
weighted avg       0.25      0.25      0.25         8



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

### **Prediction**

In [1]:
import pickle

In [4]:
intents = json.loads(open('intents.json').read())

In [6]:
from keras.models import load_model
from transformers import AutoTokenizer, TFBertModel
# load model
model = load_model('chatbot_BERT_model.h5', custom_objects={'TFBertModel': TFBertModel})

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [7]:
model

In [26]:
def prediction(text):
  x_val = tokenizer(
    text=text,
    add_special_tokens=True,
    max_length=9,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
  
  validation = model.predict({'input_ids': x_val['input_ids'], 'attention_mask': x_val['attention_mask']})
  tag_index = np.where(validation[0] == max(validation[0]))[0][0]
  result = Y.columns[tag_index]
  return result

In [27]:
prediction('how are you?')

'greeting'

In [28]:
def get_response(text):
  tag = prediction(text)
  for data in intents['intents']:
    if data['tag'] == tag:
      result = random.choice(data['responses'])
      break
  return result

In [29]:
get_response('hi')

'Hello, thanks for asking'

In [ ]:
while True:
	message = input("Enter a message:")
	res = get_response(message)
	print(res)

Enter a message:hi
Hello, thanks for asking
Enter a message:what is your name
you can Call me Neural!
Enter a message:how are you
Good to see you again
Enter a message:how old are you
21 years!
Enter a message:ok
Hi there, how can I help?
Enter a message:what can you do for me?
Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
Enter a message:thanks
Any time!
Enter a message:bye
Bye! Come back again soon.
